## Class Test to understand __get_item__



In [ ]:


import numpy as np

class MyMedicalDataset:
    def __init__(self, image_paths, labels, transforms=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transforms = transforms

    def __len__(self):
        
        return len(self.image_paths)

    def __getitem__(self, idx):
       
        # When you do dataset[i], this method is called with idx = i

        image_path = self.image_paths[idx]
        label = self.labels[idx]

        # In a real scenario, you'd load the image (e.g., DICOM with pydicom/MONAI LoadImage)
        # For simplicity, let's pretend we load a dummy image
        image_data = np.random.rand(64, 64) # Simulate loading an image

        # Apply transforms if provided (e.g., MONAI transforms)
        if self.transforms:
            # MONAI transforms expect dictionaries
            data_dict = {"image": image_data, "label": label}
            data_dict = self.transforms(data_dict)
            image_data = data_dict["image"]
            label = data_dict["label"]

        return {"image": image_data, "label": label}

# --- Usage Example ---
image_files = ["img1.dcm", "img2.dcm", "img3.dcm"]
labels_data = [0, 1, 0]

# Imagine some MONAI transforms here
from monai.transforms import RandFlip, Compose, AsDiscrete, EnsureChannelFirstd

 #my_transforms = Compose([
#     EnsureChannelFirstd(keys="image"),
#     RandFlip(prob=0.5, spatial_axis=None)
    
# ])


dataset = MyMedicalDataset(image_files, labels_data, transforms=None)

# This calls dataset.__getitem__(0)
first_item = dataset[0]
print(f"Type of first_item: {type(first_item)}")
print(f"Keys in first_item: {first_item.keys()}")
print(f"Shape of first_item['image']: {first_item['image'].shape}")

# This calls dataset.__len__()
print(f"Dataset length: {len(dataset)}")

# This indirectly uses __getitem__ (and __len__) for iteration
for i in range(len(dataset)):
    item = dataset[i]
    print(f"Processing item {i}")

# Test on the class itself 
print(f"images path : {dataset.image_paths}" )
print(f"labels : {dataset.labels}")
print(f"Transforms : {dataset.transforms}")

Type of first_item: <class 'dict'>
Keys in first_item: dict_keys(['image', 'label'])
Shape of first_item['image']: (64, 64)
Dataset length: 3
Processing item 0
Processing item 1
Processing item 2
images path : ['img1.dcm', 'img2.dcm', 'img3.dcm']
labels : [0, 1, 0]
Transforms : None


In [ ]:
from torch.utils.data import Dataset
import pydicom
import torch
import pandas as pd
import numpy as np
from pathlib import Path

class RSNA_DICOM_Dataset(Dataset):
    def __init__(self, csv_path, dicom_dir, transform=None):
        self.df = pd.read_csv(csv_path)
        self.dicom_dir = Path(dicom_dir)
        self.transform = transform
        self.label_cols = [ 'any', 'epidural', 'intraparenchymal', 'intraventricular', 'subarachnoid', 'subdural']
        self.loader = T.LoadImage(image_only=False)
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        dcm_path = self.dicom_dir / row['filename']
        dcm = pydicom.dcmread(str(dcm_path))

        # Load image
        image, meta = self.loader(str(dcm_path)) # Using MONAI's LoadImage

        dicom_raw = pydicom.dcmread(str(dcm_path))

        center = int(dicom_raw.WindowCenter)
        width = int(dicom_raw.WindowWidth)
        slope = float(dicom_raw.RescaleSlope)
        intercept = float(dicom_raw.RescaleIntercept)

        img_np = image.astype(np.float32)
        img_np = img_np * slope + intercept

        img_windowed = np.clip(img_np, center - width // 2, center + width // 2)
        img_norm = (img_windowed - img_windowed.min()) / (img_windowed.max() - img_windowed.min() + 1e-5)

        # ==== 4. Conversion en tenseur PyTorch + ajout canal (C, H, W)
        img_tensor = torch.from_numpy(img_norm).float().unsqueeze(0)  # [1, H, W]

        # ==== 5. Resize etc.
        if self.transform:
            img_tensor = self.transform(img_tensor)

        # ==== 6. Extraction des labels
        label = torch.tensor(row[self.label_cols].values.astype(np.float32))

        return img_tensor, label


/home/tibia/Projet_Hemorragie/hemorragie-env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from torch.utils.data import DataLoader

resize_transform = T.Resize((224, 224))

train_dataset = RSNA_DICOM_Dataset(csv_path, dicom_dir, transform=resize_transform)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4)

In [ ]:
from monai.networks.nets import DenseNet121
import torch.nn as nn

model = DenseNet121(spatial_dims=2, in_channels=1, out_channels=6).cuda()
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        images, labels = batch[0].cuda(), batch[1].cuda()
        outputs = model(images)
        loss = loss_fn(outputs, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()